# Fmrate prediction

In [1]:
import numpy as np
import uproot as ur
import matplotlib.pyplot as plt
import pandas as pd
import copy
import sys

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

sys.path.append('../')

from src.utils import train_val_test_split

%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

## Split data

In [ ]:
data_df = pd.read_csv("../data/fmrate_dataset.csv")

In [ ]:
data_df

In [ ]:
data_df.columns

In [ ]:
X = data_df[["unix_time", "fe_cosmic"]] # .drop(columns=[f"rate[{i}]" for i in range(12)])
y = data_df[[f"rate[{i}]" for i in range(12)]]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y,
                                                                      val_size=0.2,
                                                                      test_size=0.2,
                                                                      random_state=42,
                                                                      shuffle=True)

In [ ]:
[el.shape for el in [X_train, X_val, X_test, y_train, y_val, y_test]]

**We're not allowed to use the test set to make any decision !!**

## Preprocessing

- https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling
- https://youtu.be/juEOOQntrd0

## Basic Fully Connected NN

In [ ]:
estimator = MLPRegressor(hidden_layer_sizes=[100, 100], random_state=42, max_iter=500)
pipe = make_pipeline(StandardScaler(), estimator)

In [ ]:
pipe.fit(X_train.drop(columns=["unix_time"]), y_train)

In [ ]:
pred = pipe.predict(X_val.drop(columns=["unix_time"]))
pd.DataFrame(pred)

In [ ]:
y_val

### Visualization

Let's only look at `rate[0]`

In [ ]:
argsort = np.argsort(X_val["unix_time"])[::-1]
sorted_time_val = X_val["unix_time"][argsort]
sorted_y_val_r0 = y_val.loc[:, "rate[0]"][argsort]
sorted_val_r0 = pred[:, 0][argsort]

fig_val_prediction_rate_0, ax = plt.subplots()
ax.plot(sorted_time_val, sorted_y_val_r0, '-g', linewidth=0.1)
ax.plot(sorted_time_val, sorted_val_r0, '-r', linewidth=0.1)
ax.set_xlabel("Tunix [s]")
ax.set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
ax.set_title("Light curve of Rate[0]")
plt.show()

In [ ]:
low_n = 1000
high_n = 1500
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

for i, (low_n, high_n) in enumerate([(1000, 1500), (2000, 2500),
                                   (10000, 10500), (12000, 12500)]):
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_y_val_r0[low_n:high_n], '-g', linewidth=0.5)
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_val_r0[low_n:high_n], '-r', linewidth=0.5)
    axs[i//2, i%2].set_xlabel("Tunix [s]")
    axs[i//2, i%2].set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
    axs[i//2, i%2].set_title(f"Light curve Rate[0]: l={low_n}, h={high_n}")
plt.show()

In [ ]:
low_n = 1000
high_n = 1500
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

for i, (low_n, high_n) in enumerate([(1000, 1200), (2000, 2200),
                                   (10000, 10200), (12000, 12200)]):
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_y_val_r0[low_n:high_n], '-g', linewidth=0.5)
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_val_r0[low_n:high_n], '-r', linewidth=0.5)
    axs[i//2, i%2].set_xlabel("Tunix [s]")
    axs[i//2, i%2].set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
    axs[i//2, i%2].set_title(f"Light curve Rate[0]: l={low_n}, h={high_n}")
plt.show()

In [ ]:
fitted_mlp = pipe["mlpregressor"]

In [ ]:
np.sqrt(fitted_mlp.loss_), np.std(sorted_y_val_r0)

In [ ]:
fig_train_loss_per_epoch, ax = plt.subplots()

ax.plot(fitted_mlp.loss_curve_)
ax.set_title("Training loss per epoch")
plt.show()

The loss is still quiet huge

In [ ]:
fitted_mlp.coefs_

**TODO: try compare it with linear regression only using `fe_cosmic`. Also show validation loss for each epoch !**

### Residual plot
**Warning: we're gonna use the whole dataset, careful about data leakage**

In [ ]:
# predict using the whole X.. just to observe the residuals
pred_X_nn = pipe.predict(X.drop(columns=["unix_time"]))

In [ ]:
def plot_pred(X, y, pred_X, show=True, title="Light curve of Rate[0]: Prediction using whole dataset"):
    fig, ax = plt.subplots()
    ax.plot(X["unix_time"], y.loc[:, "rate[0]"], '-g', linewidth=0.1)
    ax.plot(X["unix_time"], pred_X[:, 0], '-r', linewidth=0.1)
    ax.set_xlabel("Tunix [s]")
    ax.set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
    ax.set_title(title)
    if show: plt.show()
    return fig

fig_dataset_prediction_rate_0 = plot_pred(X, y, pred_X_nn)

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.03
rect_residuals = [left, bottom, width, height]
rect_histy = [left + width + spacing, bottom, 0.2, height]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_axes(rect_residuals)
ax_histy = fig.add_axes(rect_histy, sharey=ax)

ax.plot(X["unix_time"], y.loc[:, "rate[0]"]-pred_X_nn[:, 0], '-r', linewidth=0.1)
ax.set_xlabel("Tunix [s]")
ax.set_ylabel("Residual for Rate[0]")  # Nb. photons per second (averaged over each bin)
ax.set_title("Residual plot for Rate[0]")


ax_histy.tick_params(axis="y", labelleft=False)
ax_histy.set_title("Histogram")
ax_histy.set_xlabel("Count")


_ = ax_histy.hist(y.loc[:, "rate[0]"]-pred_X_nn[:, 0], bins=100, orientation='horizontal', alpha=0.5, zorder=np.inf)


plt.show()

**TODO: try compare it with linear regression only using `fe_cosmic`. Also show validation loss for each epoch !**

In [ ]:
import seaborn as sns

def plot_pull(X, y, pred_X, show=True, 
              title="Pull plot for Rate[0]", 
              title_hist="Normalized Histogram (Density)"):
    left, width = 0.1, 0.65
    bottom, height = 0.1, 0.65
    spacing = 0.03
    rect_residuals = [left, bottom, width, height]
    rect_histy = [left + width + spacing, bottom, 0.2, height]

    fig = plt.figure(figsize=(12, 10))

    ax = fig.add_axes(rect_residuals)
    ax_histy = fig.add_axes(rect_histy, sharey=ax)

    tmp = (y.loc[:, "rate[0]"]-pred_X[:, 0])/np.sqrt(y.loc[:, "rate[0]"])
    mask = ~np.isclose(y.loc[:, "rate[0]"], 0)

    ax.plot(X["unix_time"][mask], tmp[mask], '-r', linewidth=0.1)
    ax.set_xlabel("Tunix [s]")
    ax.set_ylabel("Pull for Rate[0]")  # Nb. photons per second  (averaged over each bin)
    ax.set_title(title)



    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histy.set_title(title_hist)


    # _ = ax_histy.hist(tmp[mask], bins=100, orientation='horizontal', alpha=0.5, zorder=np.inf)
    _ = sns.histplot(data=tmp[mask].to_frame("rate[0]"),
                     y="rate[0]",
                     stat="density",
                     ax=ax_histy)
    mean = np.mean(tmp[mask])
    std = np.std(tmp[mask])
    xs = np.linspace(tmp[mask].min(), tmp[mask].max(), 255)
    f = lambda x: 1/np.sqrt(2*np.pi*std**2)*np.exp(-(x-mean)**2/(2*std**2))
    _ = ax_histy.plot(f(xs), xs, zorder=np.inf, color="m", linewidth=1, linestyle="--")

    if show: plt.show()
    return fig

fig_dataset_pull_plot_rate_0 = plot_pull(X, y, pred_X_nn)

In [ ]:
from matplotlib.scale import FuncScale


def plot(data, mean, std, transform="sqrt", show=True, title="Normalized Histogram (Density)"):
    f = lambda x, mean, std: 1/np.sqrt(2*np.pi*std**2)*np.exp(-(x-mean)**2/(2*std**2))
    
    fig, ax = plt.subplots()
#     _ = sns.histplot(data=data,
#                      x="rate[0]",
#                      stat="density",
#                      ax=ax)
    ax.hist(data, bins=500, alpha=0.5, density=True)

    xs = np.linspace(data.min(), data.max(), 255)
    ax.plot(xs, f(xs, mean, std), zorder=np.inf, color="m", linewidth=1, linestyle="--")
    ax.plot([-5*std, -5*std], [0, f(xs, mean, std).max()/36], 'r', label=r"$-5\sigma$")
    ax.plot([5*std, 5*std], [0, f(xs, mean, std).max()/36], 'g', label=r"$+5\sigma$")
    ax.legend()
    if transform == "sqrt":
        ax.set_yscale(FuncScale(0, (lambda x: np.sqrt(x), lambda x: np.power(x, 2))))
        if title is not None: ax.set_title("Sqrt Normalized Histogram (Density)")
    else:
        ax.set_title(title)
    if show: plt.show()
    return fig

tmp = (y.loc[:, "rate[0]"]-pred_X_nn[:, 0])/np.sqrt(y.loc[:, "rate[0]"])
mask = ~np.isclose(y.loc[:, "rate[0]"], 0)
mean = np.mean(tmp[mask])
std = np.std(tmp[mask])
    
_ = plot(tmp[mask].to_frame("rate[0]"), mean, std)

In [ ]:
5*std

In [ ]:
def find_std(data):
    low = -np.inf
    high = np.inf
    prev_std = np.inf
    std = np.std(data)
    mean = np.mean(data)
    
    while ~np.isclose(prev_std, std):
        # Update interval
        low = -3*std + mean
        high = 3*std + mean
        
        prev_std = std
        std = np.std(data[(data>low) & (data<high)])
        print(mean, std, low, high)
    return mean, std

In [ ]:
new_mean, new_std = find_std(tmp[mask])

In [ ]:
std, new_std

In [ ]:
_ = plot(tmp[mask].to_frame("rate[0]"), mean, new_std)

In [ ]:
fig_dataset_reduced_pull_hist_rate_0 = plot(tmp[mask].to_frame("rate[0]"), mean, new_std, transform=None)

**Weights and biases logs**

In [ ]:
import wandb
from wandb.sklearn import plot_learning_curve

model = pipe
model_params = model.get_params()

# Wandb:
with wandb.init(project='POLAR-background-prediction', config=model_params):
    wandb.config.update({"val_size": 0.2,
                         "test_size": 0.2,
                         "val_len": X_val.shape[0],
                         "test_len": X_test.shape[0],
                         "train_len": X_train.shape[0],
                         "random_state": 42,
                         "shuffle_data": True})

    wandb.log({"validation/prediction-rate-0": fig_val_prediction_rate_0})
    wandb.log({"dataset/prediction-rate-0": fig_dataset_prediction_rate_0})
    wandb.log({"train/loss": fig_train_loss_per_epoch})
    # wandb.log({"dataset/pull-plot-rate-0": fig_dataset_pull_plot_rate_0})  # TODO: make it work with W&B 
    # wandb.log({"dataset/pull-reduced-hist-rate-0": fig_dataset_reduced_pull_hist_rate_0})  # TODO: same as above
    # The last one doesn't work due to seaborn histplot and my vlines. It seems they can't convert matplotlib to pyplot correctly

## Linear regression using `fe_cosmic`, to predict just `rate[0]`

In [ ]:
result = stats.linregress(X_train["fe_cosmic"], y_train["rate[0]"])

print(f"Slope a: {result.slope}, Intercept b: {result.intercept},\nR^2: {result.rvalue**2}, p-value: {result.pvalue}")
print("\n"+f"std_a: {result.stderr}, std_b: {result.intercept_stderr}")

In [ ]:
pred_linregress = X_test["fe_cosmic"]*result.slope + result.intercept

### Visualization

Let's only look at `rate[0]`

In [ ]:
# argsort = np.argsort(X_val["unix_time"])[::-1]
# sorted_time_val = X_val["unix_time"][argsort]
# sorted_y_val_r0 = y_val.loc[:, "rate[0]"][argsort]
sorted_val_r0_linregress = pred_linregress[argsort]

plt.plot(sorted_time_val, sorted_y_val_r0, '-g', linewidth=0.1)
plt.plot(sorted_time_val, sorted_val_r0_linregress, '-r', linewidth=0.1)
plt.xlabel("Tunix [s]")
plt.ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
plt.title("Light curve of Rate[0]")
plt.show()

In [ ]:
low_n = 1000
high_n = 1500
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

for i, (low_n, high_n) in enumerate([(1000, 1500), (2000, 2500),
                                   (10000, 10500), (12000, 12500)]):
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_y_val_r0[low_n:high_n], '-g', linewidth=0.5)
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_val_r0_linregress[low_n:high_n], '-r', linewidth=0.5)
    axs[i//2, i%2].set_xlabel("Tunix [s]")
    axs[i//2, i%2].set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
    axs[i//2, i%2].set_title(f"Light curve Rate[0]: l={low_n}, h={high_n}")
plt.show()

In [ ]:
low_n = 1000
high_n = 1500
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

for i, (low_n, high_n) in enumerate([(1000, 1200), (2000, 2200),
                                   (10000, 10200), (12000, 12200)]):
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_y_val_r0[low_n:high_n], '-g', linewidth=0.5)
    axs[i//2, i%2].plot(sorted_time_val[low_n:high_n], sorted_val_r0_linregress[low_n:high_n], '-r', linewidth=0.5)
    axs[i//2, i%2].set_xlabel("Tunix [s]")
    axs[i//2, i%2].set_ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
    axs[i//2, i%2].set_title(f"Light curve Rate[0]: l={low_n}, h={high_n}")
plt.show()

**TODO: measure the loss using linear regression instead of multi-layer perceptron**

In [ ]:
# np.sqrt(fitted_mlp.loss_), np.std(sorted_y_val_r0)

### Residual plot
**Warning: we're gonna use the whole dataset, careful about data leakage**

In [ ]:
# predict using the whole X (fe_cosmic).. just to observe the residuals
pred_X = X["fe_cosmic"]*result.slope + result.intercept

In [ ]:
plt.plot(X["unix_time"], y.loc[:, "rate[0]"], '-g', linewidth=0.1)
plt.plot(X["unix_time"], pred_X, '-r', linewidth=0.1)
plt.xlabel("Tunix [s]")
plt.ylabel("Avg Rate[0] [Hz]")  # Nb. photons per second (averaged over each bin)
plt.title("Light curve of Rate[0]: Prediction using whole dataset")
plt.show()

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.03
rect_residuals = [left, bottom, width, height]
rect_histy = [left + width + spacing, bottom, 0.2, height]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_axes(rect_residuals)
ax_histy = fig.add_axes(rect_histy, sharey=ax)

ax.plot(X["unix_time"], y.loc[:, "rate[0]"]-pred_X, '-r', linewidth=0.1)
ax.set_xlabel("Tunix [s]")
ax.set_ylabel("Residual for Rate[0]")  # Nb. photons per second (averaged over each bin)
ax.set_title("Residual plot for Rate[0]")



ax_histy.tick_params(axis="y", labelleft=False)
ax_histy.set_title("Histogram")
ax_histy.set_xlabel("Count")


_ = ax_histy.hist(y.loc[:, "rate[0]"]-pred_X, bins=100, orientation='horizontal', alpha=0.5, zorder=np.inf)


plt.show()

**TODO: try compare via evaluation measures with NN**